In [38]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import sys

import sys; sys.path.append(
    "/data/jerrylee/pjt/BIGFAM.v.2.0"
    )
from src import obj1, tools
import importlib

In [39]:
cohort = "simulation" # UKB, GS, simulation

# Step 1. Load FR-reg results

In [40]:
frreg_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/frreg"

In [41]:
def select_DOR_level_frreg(fns):
    fns = [fn for fn in fns if fn.split(".")[1] == "DOR"]
    return fns

In [42]:
fns = os.listdir(frreg_path)
fns = select_DOR_level_frreg(fns)
len(fns)

7290

# Step 2. Slope test


In [43]:
output_path = f"/data/jerrylee/pjt/BIGFAM.v.2.0/data/{cohort}/obj1/slope-test"
output_path

'/data/jerrylee/pjt/BIGFAM.v.2.0/data/simulation/obj1/slope-test'

In [44]:
for ii, fn in enumerate(tqdm(fns)):
    pheno_name = fn.split(".")[0]
    # load FR-reg
    frreg_fn = f"{frreg_path}/{fn}"
    df_frreg = pd.read_csv(frreg_fn, delim_whitespace=True)
    
    # resampling FR-reg to compute CIs
    df_lmbds = obj1.resampleFrregCoefficients(df_frreg, n_resample=100)    
    
    # slope test(FRLog-reg)
    df_frlogreg = obj1.familialRelationshipLogRegression(df_lmbds)
    sig = obj1._slopeSig(df_frlogreg["slope"])
    
    # save slope test results
    savefn_slopetest = f"{output_path}/{pheno_name}"
    (tools
     .raw2long(df_frlogreg, params=["slope", "intercept"])
     .to_csv(
         f"{savefn_slopetest}.slope", 
         sep='\t', 
         index=False)
     )
    df_frlogreg.to_csv(
        f"{savefn_slopetest}.raw.slope", 
         sep='\t',
         index=False
    )

100%|██████████| 7290/7290 [1:18:10<00:00,  1.55it/s]
